## Total Energy Burned
Active energy burned and basal energy burned

In [113]:
import pandas as pd

# Load file into dataframe
folder_path = 'c:/Users/aoife/Documents/Project/DataTables/'

df_aeb = pd.read_csv(folder_path + 'active-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])
df_beb = pd.read_csv(folder_path + 'basal-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])

print(df_aeb.head())
print('----------------\n')
print(df_beb.head())

# Create a list of dataframes

energy_dfs = [df_aeb, df_beb]


                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
3  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
4  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   

                  startTime                   endTime  value  
0 2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  1.822  
1 2018-10-03 10:44:40+00:00 2018-10-03 10:45:41+00:00  0.711  
2 2018-10-03 10:45:41+00:00 2018-10-03 10:46:53+00:00  2.267  
3 2018-10-03 10:46:53+00:00 2018-10-03 10:47:54+00:00  2.569  
4 2018-10-03 10:47:54+00:00 2018-10-03 10:48:25+00:00  0.564  
----------------

                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:23+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 

In [114]:
# Create a new column which contains the difference between the start and end time

for df in energy_dfs:
    if 'startTime' and 'endTime' in df.columns:
        df['duration'] = df['endTime'] - df['startTime']



'''print(df_aeb.head())
print('----------------\n')
print(df_beb.head())'''

"print(df_aeb.head())\nprint('----------------\n')\nprint(df_beb.head())"

In [115]:
# Adjust the startTime to contain just the date

for df in energy_dfs:
    if 'startTime' in df.columns:
        df['startTime'] = pd.to_datetime(df['startTime'], format='%Y-%m-%dT%H:%M:%SZ')

        df['startDate'] = df['startTime'].dt.date

        print(df.head())
        print('----------------\n')

'''df_aeb['startTime'] = pd.to_datetime(df_aeb['startTime'], format='%Y-%m-%dT%H:%M:%SZ')

df_aeb['startDate'] = df_aeb['startTime'].dt.date

print(df_aeb.head())'''

                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
3  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
4  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   

                  startTime                   endTime  value        duration  \
0 2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  1.822 0 days 00:00:21   
1 2018-10-03 10:44:40+00:00 2018-10-03 10:45:41+00:00  0.711 0 days 00:01:01   
2 2018-10-03 10:45:41+00:00 2018-10-03 10:46:53+00:00  2.267 0 days 00:01:12   
3 2018-10-03 10:46:53+00:00 2018-10-03 10:47:54+00:00  2.569 0 days 00:01:01   
4 2018-10-03 10:47:54+00:00 2018-10-03 10:48:25+00:00  0.564 0 days 00:00:31   

    startDate  
0  2018-10-03  
1  2018-10-03  
2  2018-10-03  
3  2018-10-03  
4  2018-10-03  
--------

"df_aeb['startTime'] = pd.to_datetime(df_aeb['startTime'], format='%Y-%m-%dT%H:%M:%SZ')\n\ndf_aeb['startDate'] = df_aeb['startTime'].dt.date\n\nprint(df_aeb.head())"

In [116]:
# Get the total duration per day for each participant

for df in energy_dfs:
    if 'duration' in df.columns:
        df['duration'] = df.groupby(['participantId', 'startDate'])['duration'].transform('sum')
        df['value'] = df.groupby(['participantId', 'startDate'])['value'].transform('sum')

        # Drop duplicates
        df.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

        print(df.head())
        #print(df['value'].head())
        print('----------------\n')

'''df_aeb['duration'] = df_aeb.groupby(['participantId', 'startDate'])['duration'].transform('sum')
df_aeb['value'] = df_aeb.groupby(['participantId', 'startDate'])['value'].transform('sum')

print(df_aeb['duration'].head())'''

                            participantId                 timestamp  \
0    b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
137  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-06 12:52:29+00:00   
180  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-06 12:52:29+00:00   
435  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-07 19:39:12+00:00   
590  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-14 15:16:00+00:00   

                    startTime                   endTime    value  \
0   2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  112.882   
137 2017-09-05 23:15:40+00:00 2017-09-05 23:16:42+00:00  130.400   
180 2017-09-06 00:00:06+00:00 2017-09-06 00:00:26+00:00  195.420   
435 2017-09-07 15:48:13+00:00 2017-09-07 15:49:14+00:00   78.150   
590 2017-09-14 13:40:24+00:00 2017-09-14 13:41:27+00:00  128.663   

           duration   startDate  
0   0 days 02:15:27  2018-10-03  
137 0 days 01:18:16  2017-09-05  
180 0 days 03:24:02  2017-09-06  
435 0 days 02:26:26  2017-09

"df_aeb['duration'] = df_aeb.groupby(['participantId', 'startDate'])['duration'].transform('sum')\ndf_aeb['value'] = df_aeb.groupby(['participantId', 'startDate'])['value'].transform('sum')\n\nprint(df_aeb['duration'].head())"

In [117]:
# Calculate total energy burned and total duration per day

# Sort by participantId
df_aeb.sort_values(by=['participantId'], inplace=True)
df_beb.sort_values(by=['participantId'], inplace=True)

# Rename value columns
df_aeb.rename(columns={'value': 'active_energy'}, inplace=True)
df_beb.rename(columns={'value': 'basal_energy'}, inplace=True)

# Merge on participantId and startDate

df_energy = pd.merge(df_aeb, df_beb, on=['participantId', 'startDate'])

# Create a new column which contains the total energy burned per day

df_energy['total_energy'] = df_energy['active_energy'] + df_energy['basal_energy']

# Create a new column which contains the total duration per day

df_energy['total_duration'] = df_energy['duration_x'] + df_energy['duration_y']

# Drop unnecessary columns

df_energy.drop(columns=['timestamp_x', 'startTime_x', 'endTime_x', 'timestamp_y', 'startTime_y', 'endTime_y'], inplace=True)

print(df_energy.head())
print('----------------\n')
print(df_energy['total_duration'].head())



                          participantId  active_energy      duration_x  \
0  00fd4039-9b5e-4bbb-8295-4983a3f58371         13.815 0 days 01:26:33   
1  017702d7-e540-4e1a-ab19-ca5f1c34e70b         41.148 0 days 02:01:34   
2  01866570-f91f-4d3b-9d41-0a8cdcbfa922         32.569 0 days 02:29:05   
3  0292a3a2-5d0b-42a7-ba86-03eae9cda7d5         13.806 0 days 00:15:01   
4  048ba877-5df6-47c4-b485-d3538db9e9b4         30.735 0 days 02:14:05   

    startDate  basal_energy      duration_y  total_energy  total_duration  
0  2018-08-15       227.847 0 days 04:01:00       241.662 0 days 05:27:33  
1  2017-12-23       879.016 0 days 12:30:39       920.164 0 days 14:32:13  
2  2018-02-25       685.827 0 days 08:29:49       718.396 0 days 10:58:54  
3  2017-11-24       933.963 0 days 10:28:11       947.769 0 days 10:43:12  
4  2017-11-17       593.295 0 days 08:14:35       624.030 0 days 10:28:40  
----------------

0   0 days 05:27:33
1   0 days 14:32:13
2   0 days 10:58:54
3   0 days 10:43:12
4

## BMI

Body Mass and Height

In [118]:
# Use the groupby function to get the average height/body mass for each participant

df_bm = pd.read_csv(folder_path + 'body-mass.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])
df_h = pd.read_csv(folder_path + 'height.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])

bmi_dfs = [df_bm, df_h]

for df in bmi_dfs:
    if 'value' in df.columns:
        df['value'] = df.groupby(['participantId'])['value'].transform('mean')

        # Drop duplicates
        df.drop_duplicates(subset=['participantId'], inplace=True)

        print(df.head())
        print('----------------\n')

                          participantId                 timestamp       value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00   97.976000
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00   63.503000
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00   96.666000
3  41c683b0-f6c4-4a6a-8dd2-4b9a3dd30a32 2016-03-30 02:24:59+00:00  112.846333
4  35fbae45-0f21-45ea-8ed7-8d3361a29b86 2016-07-18 17:45:38+00:00   68.690083
----------------

                          participantId                 timestamp   value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00  1.6256
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00  1.6510
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00  1.8288
3  1cba6631-3fb8-4c18-904f-ab56fe82833d 2016-03-07 05:54:00+00:00  1.5494
4  eb2c4ff0-54e2-4c34-b73d-003e3e0427ad 2016-04-02 07:17:47+00:00  1.7018
----------------



In [119]:
# Calculate the BMI for each participant and create a new df

# Sort by participantId
df_bm.sort_values(by=['participantId'], inplace=True)
df_h.sort_values(by=['participantId'], inplace=True)

# Rename value columns
df_bm.rename(columns={'value': 'bodyMass'}, inplace=True)
df_h.rename(columns={'value': 'height'}, inplace=True)

# Merge on participantId

df_bmi = pd.merge(df_bm, df_h, on='participantId')

df_bmi['bmi'] = df_bmi['bodyMass'] / (df_bmi['height'] ** 2)

print(df_bmi.head())


                          participantId               timestamp_x  bodyMass  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174 2016-05-12 19:15:44+00:00    51.256   
1  00d6d2ee-ccea-45c7-9772-b19fd9bef2bf 2018-04-20 04:08:33+00:00    74.389   
2  00fd4039-9b5e-4bbb-8295-4983a3f58371 2018-08-15 04:32:59+00:00    59.874   
3  0113e483-0fc8-4892-a4fc-0b2f3820dde3 2019-03-06 19:58:35+00:00    68.039   
4  013d82d7-b3cc-4007-b00c-4d1d75bab9dd 2018-02-21 02:13:48+00:00    76.204   

                timestamp_y  height        bmi  
0 2016-05-12 19:15:44+00:00  1.5748  20.667783  
1 2018-04-20 04:08:33+00:00  1.9050  20.498343  
2 2018-08-15 04:32:59+00:00  1.6510  21.965653  
3 2019-03-06 19:58:35+00:00  1.7018  23.493130  
4 2018-02-21 02:13:48+00:00  1.6002  29.759747  


## Heart Rate

In [120]:

df_hr = pd.read_csv(folder_path + 'heart-rate.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])

# Convert value from seconds to minutes

# df_hr['value'] = df_hr['value'] * 60

In [121]:
# Convert startTime to datetime and extract date
df_hr['startTime'] = pd.to_datetime(df_hr['startTime'], format='%Y-%m-%dT%H:%M:%SZ')
df_hr['endTime'] = pd.to_datetime(df_hr['endTime'], format='%Y-%m-%dT%H:%M:%SZ')
df_hr['startDate'] = df_hr['startTime'].dt.date
print(df_hr.head()) 

                          participantId                 timestamp  \
0  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
1  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
2  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
3  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
4  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   

                  startTime                   endTime  value   startDate  
0 2016-03-13 13:52:58+00:00 2016-03-13 13:52:58+00:00  1.500  2016-03-13  
1 2016-03-13 13:53:00+00:00 2016-03-13 13:53:00+00:00  1.500  2016-03-13  
2 2016-03-13 13:53:01+00:00 2016-03-13 13:53:01+00:00  1.517  2016-03-13  
3 2016-03-13 13:53:02+00:00 2016-03-13 13:53:02+00:00  1.533  2016-03-13  
4 2016-03-13 13:53:03+00:00 2016-03-13 13:53:03+00:00  1.550  2016-03-13  


In [122]:


# Remove rows where participantId, startTime and endTime are duplicated

df_hr.drop_duplicates(subset=['participantId', 'startTime', 'endTime'], inplace=True)

print(df_hr.head())


                          participantId                 timestamp  \
0  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
1  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
2  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
3  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
4  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   

                  startTime                   endTime  value   startDate  
0 2016-03-13 13:52:58+00:00 2016-03-13 13:52:58+00:00  1.500  2016-03-13  
1 2016-03-13 13:53:00+00:00 2016-03-13 13:53:00+00:00  1.500  2016-03-13  
2 2016-03-13 13:53:01+00:00 2016-03-13 13:53:01+00:00  1.517  2016-03-13  
3 2016-03-13 13:53:02+00:00 2016-03-13 13:53:02+00:00  1.533  2016-03-13  
4 2016-03-13 13:53:03+00:00 2016-03-13 13:53:03+00:00  1.550  2016-03-13  


In [123]:
# Count the number of rows per person per day

df_hr['count'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('count')

# get the maximum heart rate per day
df_hr['max_hr'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('max')

# get the minimum heart rate per day
df_hr['min_hr'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('min')

# get the average heart rate per day
df_hr['value'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('mean')



# Drop duplicates
df_hr.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

print(df_hr)

                               participantId                 timestamp  \
0       06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
382     2214f4fd-1ae0-4804-8663-b01c5f6d142c 2016-03-02 19:58:02+00:00   
637     b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
651     b1406c4e-e6ac-4297-a9e4-335ca5ef04de 2019-01-24 03:34:02+00:00   
680     b1406c4e-e6ac-4297-a9e4-335ca5ef04de 2019-01-24 03:17:47+00:00   
...                                      ...                       ...   
890164  a5fa37f1-de3c-41bd-9324-c31c5244e5c8 2019-06-24 02:35:18+00:00   
890259  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99 2019-06-25 09:36:30+00:00   
890275  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99 2019-06-25 09:36:06+00:00   
890284  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99 2019-06-28 09:36:33+00:00   
890300  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99 2019-06-29 00:32:14+00:00   

                       startTime                   endTime     value  \
0      2016-03-13 13:52:58+00:00 2016-0

# Nap Tracker

Total duration

Average nap quality

In [124]:
df_nt = pd.read_csv(folder_path + 'nap-tracker.csv', usecols=['participantId', 'timestamp', 'NapDuration', 'NapQuality'], parse_dates=['timestamp'])

# calculate total nap duration per day for each participant

df_nt['timestamp'] = pd.to_datetime(df_nt['timestamp'], utc=True)
df_nt['startDate'] = df_nt['timestamp'].dt.date

# Remove rows where startDate, participantId and NapDuration are duplicated
df_nt.drop_duplicates(subset=['participantId', 'startDate', 'NapDuration'], inplace=True)

df_nt['totalNapTime'] = df_nt.groupby(['participantId', 'startDate'])['NapDuration'].transform('sum')

# Get the average nap quality per day for each participant

df_nt['avgNapQuality'] = df_nt.groupby(['participantId', 'startDate'])['NapQuality'].transform('mean')

print(df_nt.head())

                          participantId  NapDuration  NapQuality  \
0  cb4c4df0-6685-4743-a3cf-d29c3d6941c9       4500.0           4   
1  1f230684-4fa7-48ce-a7c2-6372238fe486      24708.0           4   
2  02b96d81-c9f6-4a73-a075-75a64e54d005       3300.0           4   
3  28e7a976-55e2-4ff1-8e95-7278f1ee9ec9          0.0           3   
4  1dcf514b-8610-40f3-9a78-43ef80b5b7b7       4093.0           1   

                  timestamp   startDate  totalNapTime  avgNapQuality  
0 2016-07-11 02:37:13+00:00  2016-07-11        4500.0            4.0  
1 2016-09-07 12:20:41+00:00  2016-09-07       24708.0            4.0  
2 2016-03-12 01:47:15+00:00  2016-03-12        3300.0            4.0  
3 2016-09-07 22:48:16+00:00  2016-09-07           0.0            3.0  
4 2016-09-08 13:21:58+00:00  2016-09-08        4093.0            1.0  


In [125]:
df_sc = pd.read_csv(folder_path + 'step-count.csv', usecols=['participantId', 'startTime', 'endTime', 'value'], parse_dates=['startTime', 'endTime'])

# get total step count per day for each participant

df_sc['startTime'] = pd.to_datetime(df_sc['startTime'], utc=True)
df_sc['endTime'] = pd.to_datetime(df_sc['endTime'], utc=True)
df_sc['startDate'] = df_sc['startTime'].dt.date

df_sc['totalSteps'] = df_sc.groupby(['participantId', 'startDate'])['value'].transform('sum')

# Get rid of duplicates

df_sc.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

print(df_sc.head())

                            participantId                 startTime  \
0    1f649060-680a-4c80-a551-be38ce46cb94 2016-03-22 22:07:20+00:00   
21   1f649060-680a-4c80-a551-be38ce46cb94 2016-03-23 00:00:52+00:00   
91   1f649060-680a-4c80-a551-be38ce46cb94 2016-03-24 00:07:26+00:00   
116  1f649060-680a-4c80-a551-be38ce46cb94 2016-03-29 20:22:53+00:00   
127  1f649060-680a-4c80-a551-be38ce46cb94 2016-03-30 00:06:09+00:00   

                      endTime  value   startDate  totalSteps  
0   2016-03-22 22:08:20+00:00    118  2016-03-22       10017  
21  2016-03-23 00:05:11+00:00      9  2016-03-23       14752  
91  2016-03-24 00:08:26+00:00     92  2016-03-24       15744  
116 2016-03-29 20:27:54+00:00    163  2016-03-29       14477  
127 2016-03-30 00:08:39+00:00     45  2016-03-30       12176  


# Sleep Quality

For sleep quality, I will change the timestamp to just include the date, and ensure there are no duplicates

In [126]:
df_sq = pd.read_csv(folder_path + 'sleep-quality-checker.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])

df_sq['timestamp'] = pd.to_datetime(df_sq['timestamp'], utc=True)

df_sq['startDate'] = df_sq['timestamp'].dt.date

# drop duplicates
df_sq.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

# Sampling Participants

The amount of rows in these dataframes are still too large, so I will randomly sample a portion of the participants.

I will first of all find the participants which have data in each dataframe, on the same days (except BMI).

Then I use a probability to keep a certain portion of the data.

In [127]:
import numpy as np

# Create a list of DataFrames
dfs_list = [df_bmi, df_energy, df_hr, df_nt, df_sc, df_sq]	

# Determine the set of participants common to all DataFrames
common_participants = set(dfs_list[0]['participantId'].unique())

for df in dfs_list[1:]:
    df_filtered = df[df['participantId'].isin(common_participants)]

    common_startDates = set(df_filtered['startDate'].unique())

    # Make sure common_participants is updated to only include participants with common startDates
    common_participants = common_participants.intersection(set(df_filtered[df_filtered['startDate'].isin(common_startDates)]['participantId'].unique()))

# Check that the common participants are also present in the BMI DataFrame, and remove any that are not
common_participants = [participant for participant in common_participants if participant in set(df_bmi['participantId'])]



print(common_participants)

# Probability of keeping a participant
keep_probability = 0.6

# Determine which participants to keep based on the probability
participants_to_keep = np.random.choice(list(common_participants), size=int(len(common_participants) * keep_probability), replace=False)

# Filter each original DataFrame to keep only the common participants
filtered_dfs = [df[df['participantId'].isin(participants_to_keep)] for df in dfs_list]

# Checking values
for i, df in enumerate(filtered_dfs):

    print(f'Columns of filtered_df[{i}]: {df.columns}')

    unique_participants = df['participantId'].unique()
    print(f"Unique participants in DataFrame {i}:")
    print(unique_participants)
    print(f"Number of unique participants: {len(unique_participants)}")
    print("\n" + "-"*30 + "\n")

    print(f"DataFrame {i}:")
    #print(df)
    print(f"Number of rows: {len(df)}")
    print("\n" + "-"*30 + "\n")




['fbd4d6f8-4471-4757-9a94-b187ce9b71b2', 'dc21e43d-f04b-4b41-9dff-454ad433a4ec', 'c58a5380-9d00-4c26-b7ee-a2460378e96a', 'd96f3530-9573-4e62-ba04-e669013e32f4', '6c52af7f-cef0-4ad2-bad4-74d3d99daee8', 'cce2d219-8dea-4737-8037-027186f5555b', 'd779a516-f476-4094-9495-21acd7688a4e', '6c95a170-a888-4f60-bf2e-ff433e68d901', '86bd116f-9074-4200-a481-52b351f0cf53', '56a4512f-5929-4e11-9dea-faa958a3e6dd', 'ed42ffc9-6f83-4c12-b3c8-852be9ed049a', '55d49397-8aae-4c41-be90-a892217a9d1d', 'a42a874b-1895-401e-8ab7-aaac4b1e0501', 'be4f8cb4-6507-4ed9-918a-412f27771239', '29e0bb9f-d896-4a30-af5f-5928f925fda9', '692b3029-d138-42ed-8ef2-a323206b69ca', '46a1eb30-cc2a-4a13-9bc9-56feb6b7a490', 'd6ccf53c-7720-453c-a378-47bea7248d6a', '977c3462-62c6-43b8-adcf-145e7bc6d16e', 'b55a1704-aa26-4592-9b83-6f74f8dd435e', 'ee0cb856-6ad2-433b-94b1-99ff01807551', 'd02538ab-c95e-4ba3-99ee-65dc48ec76a5', 'ff2dd69c-be97-4a41-8b0a-8711988f9155', 'c29124de-98a6-41f7-8385-a8a51d98b7ac', '115891f3-eabc-47c3-aaac-3531ffb5d5aa',

# Save to CSV's

In [128]:
# Save the filtered DataFrames to new CSV files

for i, df in enumerate(filtered_dfs):
    if i == 0:
        df.to_csv(f'bmi.csv', index=False)
        print(f"DataFrame {i} has been saved to bmi.csv")
    elif i == 1:
        df.to_csv(f'energy.csv', index=False)
        print(f"DataFrame {i} has been saved to aeb.csv")
    elif i == 2:
        df.to_csv(f'hr.csv', index=False)
        print(f"DataFrame {i} has been saved to hr.csv")
    elif i == 3:
        df.to_csv(f'nt.csv', index=False)
        print(f"DataFrame {i} has been saved to nt.csv")
    elif i == 4:
        df.to_csv(f'sc.csv', index=False)
        print(f"DataFrame {i} has been saved to sc.csv")
    elif i == 5:
        df.to_csv(f'sq.csv', index=False)
        print(f"DataFrame {i} has been saved to sq.csv")
        
    

DataFrame 0 has been saved to bmi.csv
DataFrame 1 has been saved to aeb.csv
DataFrame 2 has been saved to hr.csv
DataFrame 3 has been saved to nt.csv
DataFrame 4 has been saved to sc.csv
DataFrame 5 has been saved to sq.csv
